In [232]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import ast

# 1. Looking to Initial DB

<a id="section1"></a>

In [233]:
try:
    df = pd.read_csv(r'Initial DB\books.csv')
except pd.errors.ParserError as e:
    print(f"Error: {e}")

# Error on the data so we went to correct the data in the csv file


Error: Error tokenizing data. C error: Expected 12 fields in line 3350, saw 13



- Error: Error tokenizing data. C error: Expected 12 fields in line 3350, saw 13
- Error: Error tokenizing data. C error: Expected 12 fields in line 4704, saw 13
- Error: Error tokenizing data. C error: Expected 12 fields in line 5879, saw 13
- Error: Error tokenizing data. C error: Expected 12 fields in line 8981, saw 13

Those error are all because some author fields contains ",", so it kill the shape of the csv file.
We solve that by hand and the new corrected file is located in "Initial\corrected_books.csv"


# 2. Exploring DataBases

In [234]:
df = pd.read_csv(r'Initial DB\corrected_books.csv')
df_author = pd.read_csv(r'Feature DB\author_info.csv')
df_genre = pd.read_csv(r'Feature DB\books_genre.csv')
df_info = pd.read_csv(r'Feature DB\books_info.csv')

In [235]:

df.head()

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


In [236]:
df_author.head()

,average_rating,author_id,text_reviews_count,name,ratings_count
0,3.98,604031,7,Ronald J. Fields,49
1,4.08,626222,28716,Anita Diamant,546796
2,3.92,10333,5075,Barbara Hambly,122118
3,3.68,9212,36262,Jennifer Weiner,888522
4,3.82,149918,96,Nigel Pennick,1740


In [237]:
df.isna().sum()

bookID                0
title                 0
authors               0
average_rating        0
isbn                  0
isbn13                0
language_code         0
  num_pages           0
ratings_count         0
text_reviews_count    0
publication_date      0
publisher             0
dtype: int64

In [238]:
df.describe()

,bookID,average_rating,isbn13,num_pages,ratings_count,text_reviews_count
count,11127.000000,11127.000000,1.112700e+04,11127.000000,1.112700e+04,11127.000000
mean,21310.938887,3.933631,9.759888e+12,336.376921,1.793641e+04,541.854498
std,13093.358023,0.352445,4.428964e+11,241.127305,1.124794e+05,2576.176608
min,1.000000,0.000000,8.987060e+09,0.000000,0.000000e+00,0.000000
25%,10287.000000,3.770000,9.780345e+12,192.000000,1.040000e+02,9.000000
50%,20287.000000,3.960000,9.780586e+12,299.000000,7.450000e+02,46.000000
75%,32104.500000,4.135000,9.780873e+12,416.000000,4.993500e+03,237.500000
max,45641.000000,5.000000,9.790008e+12,6576.000000,4.597666e+06,94265.000000


In [239]:
df_author = df_author.dropna()
df_author.head()

,average_rating,author_id,text_reviews_count,name,ratings_count
0,3.98,604031,7,Ronald J. Fields,49
1,4.08,626222,28716,Anita Diamant,546796
2,3.92,10333,5075,Barbara Hambly,122118
3,3.68,9212,36262,Jennifer Weiner,888522
4,3.82,149918,96,Nigel Pennick,1740


In [240]:
df_info = df_info.dropna()
df_info.head()

,book_id,country_code,authors,isbn13
0,38568,US,"[{'author_id': '17059', 'role': ''}]",9780060773755
1,38562,US,"[{'author_id': '17059', 'role': ''}]",9780060774073
2,38564,US,"[{'author_id': '21701', 'role': ''}]",9780425209424
3,39307,US,"[{'author_id': '1356336', 'role': ''}]",9780312940027
4,23717,US,"[{'author_id': '12948', 'role': ''}]",9781569312063


In [241]:
df_genre = df_genre.dropna()

df_genre.head()

,book_id,genres
0,38568,"{'fantasy, paranormal': 1907, 'romance': 1598,..."
1,38562,"{'fantasy, paranormal': 1002, 'romance': 896, ..."
2,38564,"{'young-adult': 315, 'fantasy, paranormal': 27..."
3,39307,"{'fantasy, paranormal': 128, 'romance': 40, 'f..."
4,23717,"{'fantasy, paranormal': 3, 'fiction': 2, 'comi..."


<H1>Join the datasets</h1>

Verify unique value on the join columns

In [242]:
values_to_keep = df['isbn13'].unique()

#Filter df_info to keep only rows where column1 values are in values_to_keep
df_info = df_info[df_info['isbn13'].isin(values_to_keep)]

df_info.to_csv(r"Feature DB\books_info.csv",index=False)

In [243]:
values_to_keep = df_info['book_id'].unique()

# Filter df_info to keep only rows where column1 values are in values_to_keep
df_genre = df_genre[df_genre['book_id'].isin(values_to_keep)]

df_genre.to_csv(r"Feature DB\books_genre.csv",index=False)

<H2>join books.csv and books_info.csv in final_df</h2>

In [244]:
df['isbn13'] = df['isbn13'].astype(str)
df_info['isbn13'] = df_info['isbn13'].astype(str)

final_df = pd.merge(df, df_info, on = 'isbn13', how = 'inner')
final_df.head()

,bookID,title,authors_x,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,book_id,country_code,authors_y
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,1,US,"[{'author_id': '1077326', 'role': ''}, {'autho..."
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,2,US,"[{'author_id': '1077326', 'role': ''}, {'autho..."
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic,4,US,"[{'author_id': '1077326', 'role': ''}]"
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,5,US,"[{'author_id': '1077326', 'role': ''}, {'autho..."
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic,8,US,"[{'author_id': '1077326', 'role': ''}, {'autho..."


<H2>join final_df and author_info.csv in final_df</h2>

In [245]:
final_df = pd.merge(final_df, df_genre, on = 'book_id', how = 'inner')
def extract_keys(cell):
    try:
        # Convert the string representation of a dictionary to a dictionary object
        dictionary = ast.literal_eval(cell)
        # Return the keys if the input is a dictionary
        return list(dictionary.keys())
    except (ValueError, SyntaxError):
        # Return an empty list if the input is not a valid dictionary
        return []

# Apply the function to each cell of the DataFrame
final_df['genres'] = final_df['genres'].apply(extract_keys)
final_df.head()

,bookID,title,authors_x,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,book_id,country_code,authors_y,genres
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,1,US,"[{'author_id': '1077326', 'role': ''}, {'autho...","[fantasy, paranormal, young-adult, fiction, ch..."
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,2,US,"[{'author_id': '1077326', 'role': ''}, {'autho...","[fantasy, paranormal, children, fiction, young..."
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic,4,US,"[{'author_id': '1077326', 'role': ''}]","[fantasy, paranormal, young-adult, fiction, ch..."
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.,5,US,"[{'author_id': '1077326', 'role': ''}, {'autho...","[fiction, children, fantasy, paranormal, young..."
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic,8,US,"[{'author_id': '1077326', 'role': ''}, {'autho...","[fantasy, paranormal, young-adult, fiction, ch..."


In [246]:
final_df.to_csv(r"Feature DB\improved_dataset.csv", index=False)

# 3. Featuring of improved_dataset.csv

<h2>Author column featuring - Weighted Mean of each rates of authors how wrote the book</H2>
The code below is calling a python script contained in Tools folder.
For a mystic reason it was not possible to run it in the notebook, the call is followed by three paths as argument that are needed to get the datasets in the script 

In [247]:
!python Tools\weighted_mean_for_author_rates.py "Feature DB\improved_dataset.csv" "Feature DB\author_info.csv" "Feature DB\improved_dataset.csv"
